In [ ]:
from google.colab import drive

drive.mount('./gdrive', force_remount=True)
%cd './gdrive/My Drive/Colab Notebooks/cubricks'

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

seed = 42

In [ ]:
def plot_countplot(df, cols, title=None, rotation=0):
    for col in cols:
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 2))
        g = sns.countplot(np.squeeze(df[col]), ax=ax)
        g.set_xticklabels(labels=g.get_xticklabels(), rotation=rotation)
        g.set_title(title)

def plot_confuncion_matrix(y_test, predict, title='Confusion Matrix', report=True):
    if report: print(classification_report(y_test, y_predict))
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 8))
    g = sns.heatmap(confusion_matrix(y_test, predict), fmt='d', square=True, annot=True, cmap='Blues', ax=ax)
    g.set_title(title)

def plot_feature_importance(features, importances):
    features = np.array(features)
    indices = np.argsort(importances)[::-1]
    print(f'Feature ranking:')

    for f in range(len(features)):
        print(f'{importances[indices[f]]}\t{features[indices[f]]}')

    plt.figure(figsize=(10, 8))
    plt.barh(range(len(features)), importances[indices])
    plt.yticks(range(len(features)), features[indices])
    plt.title('Feature Importance')
    plt.gca().invert_yaxis()
    plt.show()

def setup_buckets(df, col, bins, sufix='Bucket'):
    bins = [-np.inf] + bins + [np.inf]
    labels = [f'{bins[i]} to {bins[i+1]-1}' for i in range(len(bins[:-1]))]
    df[col + sufix + 'Category'] = pd.cut(df[col], bins=bins, labels=labels, right=False, include_lowest=True)
    df[[col + sufix]] = df[[col + sufix + 'Category']].apply(lambda x: pd.Categorical(x, ordered=True).codes)
    return df

def split_data_month_window(df, col, date, month_window):
    date_0 = pd.to_datetime(date)
    date_1 = date_0 - pd.DateOffset(months=month_window)
    date_2 = date_0 + pd.DateOffset(months=1)

    train = df[(df[col] >= date_1) & (df[col] < date_0)]
    test = df[(df[col] >= date_0) & (df[col] < date_2)]

    train.reset_index(drop=True, inplace=True)
    test.reset_index(drop=True, inplace=True)
    return train, test

def classifier_predict(classifier, x_test, threshold=0.5):
    return (classifier.predict_proba(x_test)[:,1] >= threshold).astype('int')

In [ ]:
### Read dataset ###
df = pd.read_csv('InvoicedDocuments_v5.csv', sep=';', na_values=['N/I'], parse_dates=['DocumentDate', 'DueDate', 'ClearingDate'])

### First filters ###
df.dropna(subset=['ClearingDate', 'PaymentTerms'], inplace=True)
df = df[(df['DueDate'] > df['DocumentDate']) & (df['ClearingDate'] > df['DocumentDate'])]
df = df[df['DocumentDate'] >= '2018-03-01']

df.sort_values(by=['DocumentDate'], ascending=True, ignore_index=True, inplace=True)
df.reset_index(drop=True, inplace=True)

### Fix numerical columns ###
for amount, count in zip(['InvoicedAmount', 'PaidAmount', 'PaidPastAmount', 'OpenAmount', 'PastDueAmount'],
                         ['InvoicedDocuments', 'PaidDocuments', 'PaidPastDocuments', 'OpenDocuments', 'PastDueDocuments']):
    df[amount] = df[amount] / df[count]
    df[[amount, count]] = df[[amount, count]].fillna(0)

avg_cols = ['AvgDSOPastDueDocuments', 'AvgPastDueDays']
df[avg_cols] = df[avg_cols].fillna(0)

### Extract date information ###
date_int = lambda x: x.astype('timedelta64[D]').astype(int)

for col in ['DocumentDate', 'DueDate']:
    df[col + 'DayOfYear'] = pd.DatetimeIndex(df[col]).dayofyear
    df[col + 'Month'] = pd.DatetimeIndex(df[col]).month
    df[col + 'Day'] = pd.DatetimeIndex(df[col]).day
    df[col + 'WeekDay'] = pd.DatetimeIndex(df[col]).weekday

### Days to DueDate ###
df['DocumentDateToDueDate'] = date_int(df['DueDate'] - df['DocumentDate'])
### Days to MonthEnd ###
df['DocumentDateToMonthEnd'] = date_int((df['DocumentDate'] + pd.offsets.MonthEnd(1)) - df['DocumentDate'])
df['DueDateToMonthEnd'] = date_int((df['DueDate'] + pd.offsets.MonthEnd(1)) - df['DueDate'])
### Days to ClearingDate ###
df['DueDateToClearingDate'] = date_int(df['ClearingDate'] - df['DueDate'])
df['DocumentDateToClearingDate'] = date_int(df['ClearingDate'] - df['DocumentDate'])


### Categorical columns ###
df['CustomerRegion'].fillna(df['CustomerRegion'].value_counts().idxmax(), inplace=True)

category_cols = ['CompanyKey', 'CustomerKey', 'CorporateDivision', 'CustomerRegion', 'PaymentTerms',]
df[category_cols] = df[category_cols].apply(lambda x: pd.Categorical(x, ordered=False).codes)

# for col in category_cols:
#     value_counts = df[col].value_counts()
#     # index_list = value_counts[int(len(value_counts) * 0.75):].index
#     index_list = value_counts[value_counts < value_counts.mean() * 0.001].index
#     df[col] = df[col].apply(lambda x: 'Other' if x in index_list else x)

df.head()

In [ ]:
df['CustomerRegion'].value_counts().values

In [ ]:
df = setup_buckets(df, col='DueDateToClearingDate', bins=[1], sufix='Bin')
df = setup_buckets(df, col='DueDateToClearingDate', bins=[1, 4, 7], sufix='Bucket')
# df = setup_buckets(df, col='DueDateToClearingDate', bins=[-1, 1], sufix='Bin')
# df = setup_buckets(df, col='DueDateToClearingDate', bins=[-7, -4, -3, -2, -1, 0, 1, 2, 3, 5, 8], sufix='Bucket')

plot_countplot(df, cols=['DueDateToClearingDateBinCategory', 'DueDateToClearingDateBucketCategory'])

In [ ]:
from sklearn.preprocessing import RobustScaler, QuantileTransformer, PowerTransformer, Normalizer, StandardScaler


def prepare_data(train, test, y_column, x_column, num_first=None, random_state=None):
    randomize = np.arange(train.shape[0])
    np.random.seed(random_state)
    np.random.shuffle(randomize)

    y_column, x_column = np.array(y_column), np.array(x_column)
    x_train, y_train = train[x_column].values[randomize], train[y_column].values[randomize]
    x_test, y_test = test[x_column].values, test[y_column].values

    if num_first is not None:
        x_train[:,:num_first] = np.vectorize(np.log)(x_train[:,:num_first] + 1)
        x_test[:,:num_first] = np.vectorize(np.log)(x_test[:,:num_first] + 1)

        qt = RobustScaler(quantile_range=(5, 95))
        qt.fit(np.concatenate((x_train[:,:num_first], x_test[:,:num_first])), np.concatenate((y_train, y_test)))

        x_train[:,:num_first] = qt.transform(x_train[:,:num_first])
        x_test[:,:num_first] = qt.transform(x_test[:,:num_first])

    return x_train, y_train, x_test, y_test


y_column = ['DueDateToClearingDateBin']
x_column = [
            'AvgPastDueDays',
            'AvgDSOPastDueDocuments',
            # 'PaidDocuments',
            # 'PaidAmount',
            # 'InvoicedDocuments',
            # 'InvoicedAmount',
            # 'OpenDocuments',
            # 'OpenAmount',
            # 'PaidPastDocuments',
            'PastDueAmount',
            'PaidPastAmount',
            # 'PastDueDocuments',
            'DocumentAmount',
            'DocumentDateToDueDate',
                'CompanyKey',
                'PaymentTerms',
                'CorporateDivision',
                'CustomerKey',
                'CustomerRegion',
            'DocumentDateDay',
            'DocumentDateWeekDay',
            'DocumentDateDayOfYear',
            # 'DocumentDateToMonthEnd',
            # 'DocumentDateMonth',
            'DueDateDay',
            'DueDateWeekDay',
            'DueDateDayOfYear',
            # 'DueDateToMonthEnd',
            # 'DueDateMonth',
            ]


train, test = split_data_month_window(df, col='DueDate', date='2020-08-01', month_window=12)

x_train, y_train, x_test, y_test = prepare_data(train, test, y_column, x_column, num_first=6, random_state=seed)

plot_countplot(train, cols=['DueDateToClearingDateBinCategory'], title='Train')
plot_countplot(test, cols=['DueDateToClearingDateBinCategory'], title='Test')

In [ ]:
clf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=seed, n_jobs=-1)
clf.fit(x_train, np.squeeze(y_train))

y_predict = classifier_predict(clf, x_test, threshold=0.5)

plot_confuncion_matrix(test[y_column].values, y_predict)

In [ ]:
y_predict = classifier_predict(clf, x_test, threshold=0.9999)

plot_confuncion_matrix(test[y_column].values, y_predict)

In [ ]:
plot_feature_importance(x_column, clf.feature_importances_)

In [ ]:
print(test[test['DueDateToClearingDateBin'] == 1]['DocumentAmount'].max())
print(test[y_predict == 1]['DocumentAmount'].max(), '\n')

print(test[test['DueDateToClearingDateBin'] == 1]['DocumentAmount'].describe(), '\n')
print(test[y_predict == 1]['DocumentAmount'].describe())